In [2]:
# from huggingface_hub import notebook_login
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from datasets import load_dataset, DatasetDict, concatenate_datasets
from datasets import Audio
from transformers import WhisperForConditionalGeneration
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer, pipeline
from accelerate import PartialState, prepare_pippy
from accelerate.utils import set_seed

from torch.distributed.pipelining import Pipe

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# notebook_login()

In [3]:
model_name = "openai/whisper-small"
cache_dir = '/mnt/8THDD0/storage/huggingface'
set_seed(42)
target_lang = 'zh'
# model_name = "openai/whisper-large-v3-turbo"
output_dir = "/mnt/8THDD0/storage/train/output/"+ model_name.split('/')[1] +"-" + target_lang

In [4]:
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="zh", task="transcribe", cache_dir=cache_dir)

In [ ]:
pipe = pipeline(task="automatic-speech-recognition", model=output_dir+'/checkpoint-30000', tokenizer=tokenizer)
try:
    pipe.save_pretrained(output_dir + '/final')
except:
    pipe.save_pretrained(output_dir + '/result')

In [ ]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

# torch.setnum_threads(1)
common_voice = DatasetDict()

common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "yue", split="test")

common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

processor = WhisperProcessor.from_pretrained("/mnt/8THDD0/storage/train/output/whisper-small-zh/final/")
model = WhisperForConditionalGeneration.from_pretrained("/mnt/8THDD0/storage/train/output/whisper-small-zh/final/").to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = common_voice.map(map_to_pred)


In [ ]:
cer = load("cer")
print(100 * cer.compute(references=result['test']['reference'], predictions=result['test']["prediction"]))

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import pipeline
from huggingface_hub import notebook_login
import os

# Define the path to your local model directory
model_path = '/mnt/8THDD0/storage/train/output/whisper-small-zh/final/'

# Define the name of your repository on Hugging Face (must include username)
repo_name = 'JackyHoCL/whisper-small-cantonese-yue-english'

# Push the model to the repository
pipeline("automatic-speech-recognition", model=model_path).push_to_hub(repo_name)

print(f"Model successfully uploaded to {repo_name}")


In [ ]:
!ct2-transformers-converter --model JackyHoCL/whisper-small-cantonese-yue-english --output_dir whisper-small-cantonese-yue-english-ct2 --copy_files tokenizer.json preprocessor_config.json --quantization float16